In [ ]:
import numpy as np

import datamanager
import torch
import pathlib

config = datamanager.HyFluidNeRFDataManagerConfig(dataparser=datamanager.HyFluidDataParserConfig())
config.dataparser.data = pathlib.Path("../data/ScalarReal")
manager: datamanager.HyFluidNeRFDataManager = config.setup(device=torch.device("cuda"))

In [3]:
ray_bundle, batch = manager.next_train()
print(ray_bundle.shape)
print(batch['image'].shape)
print(batch['indices'].shape)

torch.Size([1024])
torch.Size([1024, 3])
torch.Size([1024, 3])


In [4]:
from nerfstudio.model_components.ray_samplers import UniformSampler
from nerfstudio.model_components.scene_colliders import NearFarCollider
from nerfstudio.cameras.rays import RaySamples

collider = NearFarCollider(near_plane=1.1, far_plane=1.5)
sampler_uniform = UniformSampler(num_samples=192)

collider(ray_bundle)
ray_samples_uniform: RaySamples = sampler_uniform(ray_bundle)

print(ray_samples_uniform.shape)

torch.Size([1024, 192])


In [5]:
start_positions = ray_samples_uniform.frustums.get_start_positions()
positions = ray_samples_uniform.frustums.get_positions()

print(start_positions.shape)
print(positions.shape)

torch.Size([1024, 192, 3])
torch.Size([1024, 192, 3])


In [6]:
np.savez_compressed(
    "positions.npz",
    start_positions=start_positions.cpu().numpy(),
    positions=positions.cpu().numpy()
)

In [17]:
image_indices = batch['indices'][:, 0]
all_frames = manager.train_dataset.metadata['all_frames']
frames = datamanager.image_idx_to_frame(image_indices, all_frames)
frames_expanded = frames.to(positions.device).view(positions.shape[0], 1, 1).expand(-1, positions.shape[1], -1)
xyzt = torch.cat((positions, frames_expanded), dim=-1)

print(all_frames)
print(image_indices)
print(frames)
print(frames.shape)
print(xyzt.shape)

[120, 120, 120, 120]
tensor([269.1974, 215.7328, 249.2552,  ..., 107.3740, 238.4813,  85.5473])
tensor([ 29.1974,  95.7328,   9.2552,  ..., 107.3740, 118.4813,  85.5473])
torch.Size([1024])
